#### This notebook performs task 2.3 of CRISP-DM on our King County housing dataset

In [ ]:
# Things to plot
#   1. 2 or 3 distributions of some simple relationships,
#      just to give a taste of the data.
#   2. Factor SVD to find out how the data is distributed
#   3. Measure to what extent the data is randomly clusterable
# Won't use seaborn

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from pysrc.utils import *
df = loadPreprocessed()

#### 1. Plot simple distributions

In [ ]:
# Dont have time to learn seaborn
# Will just use matplotlib
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"]     = (12, 8)
plt.rcParams["axes.labelsize"]     = 11
plt.rcParams["axes.labelweight"]   = "bold"
plt.rcParams["scatter.edgecolors"] = "white"
plt.rcParams["lines.linewidth"]    = 0.5

In [ ]:
# Plot relation sqft_living X price

plt.xlabel("sqft_living")
plt.ylabel("price - million US$")
plt.scatter(df.sqft_living.values, df.price.values)
curLocs, _ = plt.yticks()
plt.yticks(curLocs[1:-1], [str(int(price)) for price in curLocs[1:-1] / 1e6], color='k')

plt.savefig("doc/history/cdm-23/sqft_living-price.png")
plt.show()

In [ ]:
plt.xlabel("sqft_living15")
plt.ylabel("price - million US$")
plt.scatter(df.sqft_living15.values, df.price.values)
curLocs, _ = plt.yticks()
plt.yticks(curLocs[1:-1], [str(int(price)) for price in curLocs[1:-1] / 1e6])

plt.savefig("doc/history/cdm-23/sqft_living15-price.png")
plt.show()

In [ ]:
# Categorical attribute waterfront and its effect on "price"
# Measures shown with one standard deviation error

withoutWaterDf = df[df.waterfront == 0].price
withoutWaterMean = withoutWaterDf.mean()
withoutWaterStd = withoutWaterDf.std()
withoutWaterRange = np.array([withoutWaterMean - withoutWaterStd, withoutWaterMean + withoutWaterStd], dtype=int)

withWaterDf = df[df.waterfront == 1].price
withWaterMean = withWaterDf.mean()
withWaterStd = withWaterDf.std()
withWaterRange = np.array([withWaterMean - withWaterStd, withWaterMean + withWaterStd], dtype=int)

print(f"Price without waterfront: {withoutWaterRange}")
print(f"Price with waterfront: {withWaterRange}")

In [ ]:
from pandas.plotting import scatter_matrix

In [ ]:
def plotScatterMatrix(df):
    hist_kwds = {"hist_kwds": {"edgecolor": "k", "bins": 30}}
    scatter_kwds = {"linewidth": 0.2, "edgecolor": "white"}
    scatter_matrix(df.iloc[::, :4], figsize=(13, 8), **hist_kwds, **scatter_kwds)
    plt.savefig("doc/history/cdm-23/scatter-matrix.png")
    plt.show()

plotScatterMatrix(df)

#### 2. Factor SVD to find out how the data is distributed

In [ ]:
# Maybe this reinforces the reasoning on choosing Min-Max technique

sigmas = np.linalg.svd(df, compute_uv=False)

def chooseBiggestSigmas(sigmas):
    chosenSigmas = list()
    head = 0
    while sum(chosenSigmas) / sigmas.sum() < 0.95:
        chosenSigmas.append(sigmas[head])
        head += 1

    return np.array(chosenSigmas)

chosenSigmas = chooseBiggestSigmas(sigmas)
print(f"Chosen sigmas: {chosenSigmas}")

#### 3. Test clustering null hypothesis (Gap statistic)

In [ ]:
# Is this worth doing?
# - What information is it going to give me?
# - How does that information help me achieve my data mining objective?
# What are the steps required?
# - Finish preprocessing the data (CHECK)
# - Find out how to build the gap statistic
#
# Ok. I think I know how to build the gap statistic
#
# 1. I fix the number of samples "t"
#
# 2. I have to generate samples of the dataset following the
#    Monte Carlo sampling methodology.
#
# 3. I then run K-means on each sample like 100 times, to find
#    out the best clustering for that sample.
#
# 4. Build the W similarity matrix for each cluster (1..K).
#    Calculate the W_in coefficient, which is just the sum of
#    all the values of that matrix. (should end up with 3 
#    nested loops for this part)
#
# 5. Get the mean "mu" and standard deviation of the W_in's
#
# 6. The gap statistic for K is then given as
#    gap(K) = mu - log(W_in(original dataset))
#
# It measures the difference between the null hypothesis and
#   our clustering on the original dataset, for K clusters.

In [ ]:
from pysrc.preprocessing import *
# Drop some columns for better clustering
# df = df[["price", "sqft_living", "bathrooms", "bedrooms", "floors"]]

In [ ]:
u, s, vh = np.linalg.svd(df, full_matrices=False)

In [ ]:
# So, we can reduce the dimensionality to 2 dimensions
#   (maybe it is not that good of an idea...)

dim = len(chosenSigmas)
aproxDf = u[::, :dim] @ np.diag(s[:dim])
reDf = pd.DataFrame(aproxDf, columns=["pc" + str(i) for i in range(dim)])

In [ ]:
# Normalize the data first
normalize(reDf)

# Try something to test my algorithm: separate the data
# Into two clusters. The gap statistic should go positive
reDf

In [ ]:
# Plot the reduced dimension data
plt.scatter(reDf.values[::, 0], reDf.values[::, 1], edgecolor="white", linewidth=0.5)
plt.xlabel("pc1", size=11, weight="bold")
plt.ylabel("pc2", size=11, weight="bold")
plt.savefig("doc/history/cdm-23/data2d.png")
plt.show()

#### Step 1

In [ ]:
# 1. I fix the of samples "t"
t = 5

In [ ]:
# 2. I have to generate samples of the dataset following the
#    Monte Carlo sampling methodology.
def generateSample(df, sampleSize):
    minMaxs = (df.min(), df.max())
    randSamp = np.zeros((sampSize, df.shape[1]))
    for i in range(randSamp.shape[0]):
        for j in range(randSamp.shape[1]):
            randSamp[i, j] = np.random.rand() * (minMaxs[1][j] - minMaxs[0][j]) + minMaxs[0][j]

    return randSamp

# 3. I then run K-means on each sample like 100 times, to find
#    out the best clustering for that sample.
from sklearn.cluster import KMeans

def vecEucDist(vec1, vec2):
    return np.sqrt(((vec1 - vec2) ** 2).sum(axis=1))

def runKmeans(dataMatrix, K):    
    numberOfRuns = 10
    minError = np.inf
    minModel = None
    for run in range(numberOfRuns):
        model = KMeans(n_clusters=K, random_state=None)
        model.fit(dataMatrix)
        
        # Need to be able to measure clustering error
        error = model.inertia_ # sqrError(dataMatrix, model)
        
        if error < minError:
            minError = error
            minModel = model
    
    return minModel
    
# 4. Build the W similarity matrix for each cluster (1..K).
#    Calculate the W_in coefficient, which is just the sum of
#    all the values of that matrix. (should end up with like 3 
#    nested loops for this part)
def mapClusterDict(dataMatrix, labels):
    clusters = dict()
    for cluster in np.unique(labels):
        clusters[cluster] = list()
        
    for i in range(labels.size):
        clusters[labels[i]].append(dataMatrix[i])
    
    return clusters

def buildW(cPoints):
    cSize = cPoints.shape[0]         # Cluster size
    W = np.zeros((cSize, cSize))     # W has size cSize X cSize
    for i in range(cSize):           # Go through all the points
        W[i, i:] = vecEucDist(cPoints[i], cPoints[i:])
    
    return W

def buildW_in(clusters, k):
    cPoints = np.array(clusters[k]) # Cluster points
    W_in = buildW(cPoints)
    
    return W_in
    
def calcWeightIn(dataMatrix, KMeansModel):
    centroids = KMeansModel.cluster_centers_
    labels = KMeansModel.labels_
    
    if centroids.size == 0 or labels.size == 0:
        raise AttributeError("Invalid empty model")
    
    # Build association cluster -> its points
    clusters = mapClusterDict(dataMatrix, labels)
    
    # Build W_in for each cluster
    numClusters = len(clusters.keys())
    Ws = np.zeros(numClusters)
    for k in range(numClusters):
        W_in = buildW_in(clusters, k)
        
        # W_in only has upper right entries non-null, so
        # we dont have to divide them by two
        Ws[k] = W_in.sum()
    
    return Ws

def calcAllSampleWeights(df, sampSize, K, t):
    W_ins = np.zeros(t)
    for sampIdx in range(t):
        print("Iteration ", sampIdx)

        randSamp = generateSample(df, sampSize)
        bestKMeansRun = runKmeans(randSamp, K)

        W_ins[sampIdx] = calcWeightIn(randSamp, bestKMeansRun).sum()
    
    return W_ins

def computeGapStatistic(df, sampSize, K, t):
    W_ins = calcAllSampleWeights(df, sampSize, K, t)

    # 5. Get the mean "mu" and standard deviation of the W_in's
    # use the logarithm
    mu = np.log(W_ins).mean()
    sigma = np.sqrt(((np.log(W_ins) - mu) ** 2).mean())

    # 6. The gap statistic for K is then given as
    #    gap(K) = mu - log(W_in(original dataset))
    # Have to calculate the W_in of original dataset
    bestKMeansRun = runKmeans(df.values, K)
    expectW_in = calcWeightIn(df.values, bestKMeansRun).sum()
    
    return mu - np.log(expectW_in), sigma

nIter = 9
gaps = np.zeros(nIter)
gapSigmas = np.zeros(nIter)
sampSize = reDf.shape[0]
for K in range(2, nIter + 2):
    
    gaps[K - 2], gapSigmas[K - 2] = computeGapStatistic(reDf, sampSize, K, t)
    print("\nCalculated gap statistic: ", gaps[K - 2])


In [ ]:
x, y = np.arange(2, len(gaps) + 2), gaps

plt.figure(1, figsize=(12, 8))
plt.errorbar(x, y, linewidth=1, xerr=None, yerr=2 * gapSigmas, ecolor="grey", elinewidth=4)
plt.xlabel("K", weight="bold", size=11)
plt.ylabel("gap", weight="bold", size=11)
plt.savefig("doc/history/cdm-23/gaps.png")
plt.show()

In [ ]:
# Plot the grouping on the reduced dimension
# dataframe, for optimal K

optimalK = 3

# Plot each cluster separately
bestKMeansRun = runKmeans(reDf.values, optimalK)
clusters = mapClusterDict(reDf.values, bestKMeansRun.labels_)

In [ ]:
for k in clusters.keys():
    cPoints = np.array(clusters[k])
    plt.scatter(cPoints[::, 0], cPoints[::, 1], edgecolor="white", linewidth=0.5)

# Plot the centroids above
centroids = np.array(bestKMeansRun.cluster_centers_)
plt.scatter(centroids[::, 0], centroids[::, 1], color='k', label="centroids")

plt.xlabel("pc1", size=11, weight="bold")
plt.ylabel("pc2", size=11, weight="bold")
plt.legend(labels=["cluster 1", "cluster 2", "centroids"])
plt.savefig("doc/history/cdm-23/clustering2d.png")
plt.show()

In [ ]:
# Fit original dataset with optimalK clusters

kmeansFit = runKmeans(df.values, optimalK)

In [ ]:
# Decide and use an internal cluster
#   measure, other than the gap statistic,
#   to measure the quality of the clustering
#   obtained.
#
# Options: C-Index, Dunn Index

In [ ]:
np.arange(9).reshape(3,3).flatten()

In [ ]:
W = buildW(df.values)
W_flatten = np.array(0)
for i in range(1000):
    W_flatten = np.append(W_flatten, W[i, i:])

W_flatten = np.array(W_flatten)
del W

In [ ]:
W_flatten.shape

In [ ]:
# How to compute the C-index?
#
# 1. Choose a number of distances "N_in" to be compared
#
# 2. Compute similarity matrix W
#
# 3. Compute weights W_in(already have method for that)
#
# 4. Return their mean
def getKMin(dataMatrix, K):
    mins = np.zeros(K)
    for i in range(K):
        minIndex = dataMatrix.argmin()
        mins[i] = dataMatrix[]
def cIndex(dataMatrix, KMeansModel):
    clusters = mapClusterDict(dataMatrix, KMeansModel.labels_)
    
    # Build W_in for each cluster
    numClusters = len(clusters.keys())
    cIndexes = np.zeros(numClusters)
    for k in range(numClusters):
        W_in = buildW_in(clusters, k)
        N_in = len(clusters[k])
        
        W_min = W_flat_sort[:N_in].sum()
        W_max = W_flat_sort[-N_in:].sum()
        
        print("W_in\n", W_in)
        print("N_in: ", N_in)
        print("W_min\n", W_min)
        print("W_max\n", W_max)
        
        cIndexes[k] = (W_in - W_min) / (W_max - W_min)
        
    return cIndexex.mean()
    
CI = cIndex(df.values, kmeansFit)
CI

In [ ]:
np.arange(9)[-3:]